In [5]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
import pandas as pd

In [6]:
features = pd.read_csv('../data/features.csv', index_col=0)
labels = pd.read_csv('../data/labels.csv', index_col=0)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [7]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

In [14]:
class TCN_block(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TCN_block, self).__init__()
        self.conv = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp = Chomp1d(padding)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv, self.chomp, self.relu, self.dropout)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.init_weights()

    def init_weights(self):
        self.conv.weight.data.normal_(0, 0.01)
        # if self.downsample is not None:
        #     self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x #if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

In [9]:
class TCN_Model(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size):
        super(TCN_Model, self).__init__()
        self.conv_layers = nn.Sequential(TCN_block(n_inputs, n_outputs, kernel_size),
                                    TCN_block(n_outputs, n_outputs*5, kernel_size))
        self.dense_layers = nn.Sequential(nn.Linear(n_outputs//5, n_outputs//15),
                                    nn.ReLU(),
                                    nn.Linear(n_outputs//15, n_outputs//30),
                                    nn.ReLU(),
                                    nn.Linear(n_outputs//30, n_outputs//60), 
                                    nn.ReLU(),
                                    nn.Linear(n_outputs//60, n_outputs//120), 
                                    nn.Sigmoid())
    
    def forward(self, x):
        conv_out = self.conv_layers(x)
        dense_in = torch.reshape(conv_out,(1,conv_out.size))
        return self.dense_layers(dense_in)

In [11]:
def outlier_handler(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    df = np.where(df > upr_bound, df.median(), np.where(df < lwr_bound, df.median(), df))
    return df

numeric_transformer = Pipeline([
    ('Outlier_handler', FunctionTransformer(outlier_handler)),
    ('Imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline([
    ('Imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('Binary_encoder', OneHotEncoder(sparse=False, drop='if_binary', handle_unknown='ignore'))
    # ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numeric_features = X_train.select_dtypes(['int64', 'float64']).columns
cat_cols = X_train.select_dtypes('category').columns
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, cat_cols)
])

proc = Pipeline([
    ("preprocessor", preprocessor),
    ("scaler", StandardScaler())
])

In [12]:
n_inputs = 60
n_outputs = n_inputs * 2
kernel_size = 3

model = TCN_Model(n_inputs, n_outputs, kernel_size)

In [13]:
model_input = torch.Tensor(proc.fit_transform(X_train, y_train))
model.forward(model_input)

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [120, 60, 3], but got 2-dimensional input of size [40000, 23] instead